In [ ]:
!pip install ultralytics roboflow

In [ ]:
from ultralytics import YOLO
import torch

# Check if GPU is available
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


CUDA Available: True
GPU Name: Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/coding/hoop_detect.zip -d /content/dataset

Archive:  /content/drive/MyDrive/coding/hoop_detect.zip
replace /content/dataset/classes.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/dataset/classes.txt  
  inflating: /content/dataset/notes.json  
  inflating: /content/dataset/images/00265d39-frame_0014_random_t33.60s.png  
  inflating: /content/dataset/images/0155be34-frame_0022_random_t134.40s.png  
  inflating: /content/dataset/images/01cf9050-frame_0032_random_t126.09s.png  
  inflating: /content/dataset/images/01eb6ee7-frame_0037_random_t271.44s.png  
  inflating: /content/dataset/images/0389f043-frame_0012_random_t206.27s.png  
  inflating: /content/dataset/images/044d4346-frame_0005_random_t36.80s.png  
  inflating: /content/dataset/images/04f10670-frame_0032_random_t56.60s.png  
  inflating: /content/dataset/images/05c7d388-frame_0004_random_t169.20s.png  
  inflating: /content/dataset/images/05d07ef9-frame_0010_random_t75.34s.png  
  inflating: /content/dataset/images/0741866e-frame_0036_random_t60.67s.

In [ ]:
%%bash
mkdir -p /content/dataset/images/val
mkdir -p /content/dataset/labels/val

VAL_COUNT=50  # You can change this number
VAL_IMAGES=$(find /content/dataset/images/train -name '*.jpg' | shuf -n $VAL_COUNT)

for img_path in $VAL_IMAGES; do
  base=$(basename "$img_path" .jpg)
  mv "$img_path" /content/dataset/images/val/
  mv "/content/dataset/labels/train/${base}.txt" /content/dataset/labels/val/
done

In [ ]:
%%writefile /content/dataset/data.yaml
train: /content/dataset/images/train  # Training images
val: /content/dataset/images/val  # Validation images

nc: 1
names: ["hoop"]  # Class names

Overwriting /content/dataset/data.yaml


In [ ]:
import os
print("Images in train folder:", os.listdir("/content/dataset/images/train"))

Images in train folder: []


In [ ]:
# !ls /content/dataset
!ls /content/dataset/images

00265d39-frame_0014_random_t33.60s.png
0155be34-frame_0022_random_t134.40s.png
01cf9050-frame_0032_random_t126.09s.png
01eb6ee7-frame_0037_random_t271.44s.png
0389f043-frame_0012_random_t206.27s.png
044d4346-frame_0005_random_t36.80s.png
04f10670-frame_0032_random_t56.60s.png
05c7d388-frame_0004_random_t169.20s.png
05d07ef9-frame_0010_random_t75.34s.png
0741866e-frame_0036_random_t60.67s.png
07a734ea-frame_0083_random_t1240.41s.png
08220e75-frame_0002_random_t14.77s.png
08400eb8-frame_0011_random_t80.15s.png
085a8abf-frame_0024_random_t45.40s.png
090f128a-frame_0015_random_t144.14s.png
09375d16-frame_0023_random_t33.24s.png
09b037c7-frame_0047_random_t387.22s.png
0a9e94a4-frame_0045_random_t461.46s.png
0ac079fb-frame_0005_random_t16.73s.png
0e07e0d2-frame_0027_random_t35.20s.png
0ebaf71d-frame_0016_random_t138.18s.png
0fdc431b-frame_0006_random_t28.19s.png
10cf6866-frame_0080_random_t1182.55s.png
1179cd37-frame_0032_random_t243.44s.png
117c4d81-frame_0019_random_t69.57s.png
12636419-fr

In [ ]:
import os

for root, dirs, files in os.walk("/content/dataset/labels"):
    print("📂", root)
    for file in files:
        print("   └──", file)

📂 /content/dataset/labels
   └── 79c54122-frame_0033_random_t114.01s.txt
   └── 81f1d412-frame_0034_random_t127.93s.txt
   └── b6a6b0b0-frame_0045_random_t384.30s.txt
   └── 1380af15-frame_0012_random_t56.66s.txt
   └── c05007e5-frame_0000_random_t25.61s.txt
   └── f83a320d-frame_0037_random_t286.89s.txt
   └── 2fa789fa-frame_0004_random_t156.87s.txt
   └── 669f8c29-frame_0039_random_t128.63s.txt
   └── 4815d838-frame_0030_random_t238.20s.txt
   └── 7111d2a3-frame_0006_random_t17.07s.txt
   └── d443cd5f-frame_0028_random_t52.30s.txt
   └── 96260cb5-frame_0041_random_t134.33s.txt
   └── 0a9e94a4-frame_0045_random_t461.46s.txt
   └── 34ec041b-frame_0035_random_t44.40s.txt
   └── 47a3c11c-frame_0038_random_t126.79s.txt
   └── 8f82b527-frame_0006_random_t44.46s.txt
   └── c961abf2-frame_0013_random_t61.89s.txt
   └── a5359f64-frame_0030_random_t125.99s.txt
   └── a34f2f32-frame_0040_random_t54.04s.txt
   └── 17b22daf-frame_0017_random_t36.53s.txt
   └── e5a5719c-frame_0040_random_t312.10s.

In [ ]:
import os, random, shutil

# Paths
base_dir = "/content/dataset"
image_dir = os.path.join(base_dir, "images")
label_dir = os.path.join(base_dir, "labels")
split = 0.8

# Gather all image paths
images = [f for f in os.listdir(image_dir) if f.endswith(".png")] # Images are PNG, not JPG
random.shuffle(images)

split_idx = int(len(images) * split)
train_images = images[:split_idx]
val_images = images[split_idx:]

# Create folders
for split_name in ["train", "val"]: # Use split_name to avoid shadowing built-in function split
    os.makedirs(os.path.join(image_dir, split_name), exist_ok=True)
    os.makedirs(os.path.join(label_dir, split_name), exist_ok=True)

# Move files
for img in train_images:
    shutil.move(os.path.join(image_dir, img), os.path.join(image_dir, "train", img))
    shutil.move(os.path.join(label_dir, img.replace(".png", ".txt")), os.path.join(label_dir, "train", img.replace(".png", ".txt"))) # Labels are .txt

for img in val_images:
    shutil.move(os.path.join(image_dir, img), os.path.join(image_dir, "val", img))
    shutil.move(os.path.join(label_dir, img.replace(".png", ".txt")), os.path.join(label_dir, "val", img.replace(".png", ".txt"))) # Labels are .txt

In [ ]:
import os
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

results = model.train(
    data="/content/dataset/data.yaml",
    epochs=50,  # Increase epochs for better results
    imgsz=1280,  # Image size, doesnt really matter unless its speed related
    batch=16,  # Adjust based on dataset size, butagain it doesnt really matter
    device="cuda"  # Run on Colab GPU cuz mine sucks
)
from google.colab import files
files.download('/content/runs/detect/train/weights/best.pt')

Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=Tr

train: Scanning /content/dataset/labels/train... 225 images, 2 backgrounds, 0 corrupt: 100%|██████████| 225/225 [00:00<00:00, 328.29it/s]

train: New cache created: /content/dataset/labels/train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1176.3±997.7 MB/s, size: 1510.1 KB)


val: Scanning /content/dataset/labels/val... 57 images, 1 backgrounds, 0 corrupt: 100%|██████████| 57/57 [00:00<00:00, 135.68it/s]

val: New cache created: /content/dataset/labels/val.cache


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 2 dataloader workers
Logging results to runs/detect/train4
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      7.72G      1.367      7.347      1.213         28       1280:  47%|████▋     | 7/15 [00:14<00:16,  2.09s/it]Exception ignored in: <generator object tqdm.__iter__ at 0x7fe998e5d800>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tqdm/std.py", line 1196, in __iter__
    self.close()
  File "/usr/local/lib/python3.11/dist-packages/tqdm/std.py", line 1302, in close
    self.display(pos=0)
  File "/usr/local/lib/python3.11/dist-packages/tqdm/std.py", line 1495, in display
    self.sp(self.__str__() if msg is None else msg)
  File "/usr/local/lib/python3.11/dist-packages/tqdm/std.py", line 459, in print_status
    fp_write('\r' + s + (' ' * max(last_len[0] - len_s, 0)))
  File "/usr/local/lib/python3.11/dist-packages/tqdm/std.py", line 453, in fp_write
    fp_flush()
  File "/usr/local/lib/python3.11/dist-packages/tqdm/utils.py", line 196, in inner
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/pyt

KeyboardInterrupt: 

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!cp /content/runs/detect/train/weights/best.pt /content/drive/MyDrive/hoop.pt